# mypulp

## Linear Optimization 線形最適化

In [1]:
from mypulp import *

In [2]:
model = Model('lo1')
model

lo1:
MINIMIZE
None
VARIABLES

In [3]:
x1 = model.addVar(vtype='C', name='x1')
x1

Variable: x1 lb=0.0 ub=1e+100 vtype=C X=None RC=None

In [4]:
x2 = model.addVar(name='x2')
x3 = model.addVar(name='x3')

In [5]:
x1, x2, x3

(Variable: x1 lb=0.0 ub=1e+100 vtype=C X=None RC=None,
 Variable: x2 lb=0.0 ub=1e+100 vtype=C X=None RC=None,
 Variable: x3 lb=0.0 ub=1e+100 vtype=C X=None RC=None)

In [6]:
model

lo1:
MINIMIZE
None
VARIABLES

In [7]:
model.update()

In [8]:
model.addConstr(2*x1+x2+x3 <= 60)

2*x1 + 1*x2 + 1*x3 + -60 <= 0

In [9]:
model

lo1:
MINIMIZE
None
SUBJECT TO
c_1: 2 x1 + x2 + x3 <= 60

VARIABLES
x1 <= 1e+100 Continuous
x2 <= 1e+100 Continuous
x3 <= 1e+100 Continuous

In [10]:
model.addConstr(x1+2*x2+x3 <= 60)
model.addConstr(x3<=30)
model

lo1:
MINIMIZE
None
SUBJECT TO
c_1: 2 x1 + x2 + x3 <= 60

c_2: x1 + 2 x2 + x3 <= 60

c_3: x3 <= 30

VARIABLES
x1 <= 1e+100 Continuous
x2 <= 1e+100 Continuous
x3 <= 1e+100 Continuous

In [11]:
model.setObjective(15*x1 + 18*x2 + 30*x3, GRB.MAXIMIZE)
model

lo1:
MAXIMIZE
15*x1 + 18*x2 + 30*x3 + 0
SUBJECT TO
c_1: 2 x1 + x2 + x3 <= 60

c_2: x1 + 2 x2 + x3 <= 60

c_3: x3 <= 30

VARIABLES
x1 <= 1e+100 Continuous
x2 <= 1e+100 Continuous
x3 <= 1e+100 Continuous

In [12]:
%%time
model.optimize()

CPU times: user 3.31 ms, sys: 8.06 ms, total: 11.4 ms
Wall time: 27 ms


In [13]:
model.ObjVal

1230.0

In [14]:
model.getVars()

[Variable: x1 lb=0.0 ub=1e+100 vtype=C X=10.0 RC=0.0,
 Variable: x2 lb=0.0 ub=1e+100 vtype=C X=10.0 RC=0.0,
 Variable: x3 lb=0.0 ub=1e+100 vtype=C X=30.0 RC=0.0]

In [15]:
for v in model.getVars():
    print(v.VarName, v.X)

x1 10.0
x2 10.0
x3 30.0


In [16]:
model.getConstrs()

[2*x1 + 1*x2 + 1*x3 + -60 <= 0, 1*x1 + 2*x2 + 1*x3 + -60 <= 0, 1*x3 + -30 <= 0]

In [17]:
for c in model.getConstrs():
    print(c.ConstrName, c.Pi)

c_1 4.0
c_2 7.0
c_3 19.0


## Integer Optimization 整数最適化

In [18]:
multidict

<function mypulp.multidict(dic)>

In [19]:
name, height, weight = multidict({'Taro':[1,2], 'Jiro':[3,4], 'Sabu':[5,6]})
name, height, weight

(['Taro', 'Jiro', 'Sabu'],
 {'Taro': 1, 'Jiro': 3, 'Sabu': 5},
 {'Taro': 2, 'Jiro': 4, 'Sabu': 6})

In [20]:
def example():
    J, v = multidict({1:16, 2:19, 3:23, 4:28})
    a = {(1,1):2, (1,2):3, (1,3):4, (1,4):5,
        (2,1):3000, (2,2):3500, (2,3):5100, (2,4):7200}
    I, b = multidict({1:7, 2:10000})
    return I, J, v, a, b

In [22]:
example()

([1, 2],
 [1, 2, 3, 4],
 {1: 16, 2: 19, 3: 23, 4: 28},
 {(1, 1): 2,
  (1, 2): 3,
  (1, 3): 4,
  (1, 4): 5,
  (2, 1): 3000,
  (2, 2): 3500,
  (2, 3): 5100,
  (2, 4): 7200},
 {1: 7, 2: 10000})

In [23]:
def mkp(I, J, v, a, b):
    model = Model('mkp')
    x = {}
    for j in J:
        x[j] = model.addVar(vtype='B', name='x(%d)'%j)
    model.update()
    for i in I:
        model.addConstr(quicksum(a[i,j]*x[j] for j in J)<=b[i])
    model.setObjective(quicksum(v[j]*x[j] for j in J), GRB.MAXIMIZE)
    model.update()
    return model

In [24]:
quicksum([2*x1, x2, x3])

2*x1 + 1*x2 + 1*x3 + 0

In [26]:
I, J, v, a, b = example()
model = mkp(I, J, v, a, b)
model.update()
model.write('mkp.lp')

In [27]:
model.update()
model.write('mkp.mps')

In [29]:
%%time
model.optimize()

CPU times: user 4.56 ms, sys: 8.27 ms, total: 12.8 ms
Wall time: 39.2 ms


In [30]:
model.ObjVal

42.0

In [33]:
for v in model.getVars():
    print(v.VarName, v.X)


x(1) 0.0
x(2) 1.0
x(3) 1.0
x(4) 0.0


## diet problem 栄養問題

In [8]:
model = Model('lo infeas')
x1 = model.addVar(vtype='C', name='x1')
x2 = model.addVar(vtype='C', name='x2')
model.update()
model.addConstr(x1-x2<=-1)
model.addConstr(x2-x1<=-1)
model.setObjective(x1+x2, GRB.MAXIMIZE)
model.optimize()

In [9]:
model

lo infeas:
MAXIMIZE
1*x1 + 1*x2 + 0
SUBJECT TO
c_1: x1 - x2 <= -1

c_2: - x1 + x2 <= -1

VARIABLES
x1 <= 1e+100 Continuous
x2 <= 1e+100 Continuous

In [7]:
model.Status, GRB.Status.OPTIMAL

(3, 1)

In [5]:
if model.Status == GRB.Status.OPTIMAL:
    print(model.ObjVal)
    for v in model.getVars():
        print(v.VarName, v.X)

In [13]:
F, c, n = multidict({
    'CQPounder':[360, {'Cal':556, 'Carbo':39, 'Protein':30, 'VitA':147, 'VitC':10, 'Calc':221, 'Iron':2.4}],
    'Big M':[320, {'Cal':556, 'Carbo':46, 'Protein':26, 'VitA':97, 'VitC':9, 'Calc':142, 'Iron':2.4}],
    'FFilet':[270, {'Cal':356, 'Carbo':42, 'Protein':14, 'VitA':28, 'VitC':1, 'Calc':76, 'Iron':0.7}],
    'Chicken':[290, {'Cal':431, 'Carbo':45, 'Protein':20, 'VitA':9, 'VitC':2, 'Calc':37, 'Iron':0.9}],
    'Fries':[190, {'Cal':248, 'Carbo':30, 'Protein':3, 'VitA':0, 'VitC':5, 'Calc':7, 'Iron':0.6}],
    'Milk':[170, {'Cal':138, 'Carbo':10, 'Protein':7, 'VitA':80, 'VitC':2, 'Calc':227, 'Iron':0}],
    'VegJuice':[100, {'Cal':69, 'Carbo':17, 'Protein':1, 'VitA':750, 'VitC':2, 'Calc':18, 'Iron':0}],
    
})
N, a, b = multidict({
    'Cal':[2000, 3000],
    'Carbo':[300, 375],
    'Protein':[50, 60],
    'VitA':[500, 750],
    'VitC':[85, 100],
    'Calc':[660, 900],
    'Iron':[6.0, 7.5],
    
})

In [15]:
F, c, n

(['CQPounder', 'Big M', 'FFilet', 'Chicken', 'Fries', 'Milk', 'VegJuice'],
 {'CQPounder': 360,
  'Big M': 320,
  'FFilet': 270,
  'Chicken': 290,
  'Fries': 190,
  'Milk': 170,
  'VegJuice': 100},
 {'CQPounder': {'Cal': 556,
   'Carbo': 39,
   'Protein': 30,
   'VitA': 147,
   'VitC': 10,
   'Calc': 221,
   'Iron': 2.4},
  'Big M': {'Cal': 556,
   'Carbo': 46,
   'Protein': 26,
   'VitA': 97,
   'VitC': 9,
   'Calc': 142,
   'Iron': 2.4},
  'FFilet': {'Cal': 356,
   'Carbo': 42,
   'Protein': 14,
   'VitA': 28,
   'VitC': 1,
   'Calc': 76,
   'Iron': 0.7},
  'Chicken': {'Cal': 431,
   'Carbo': 45,
   'Protein': 20,
   'VitA': 9,
   'VitC': 2,
   'Calc': 37,
   'Iron': 0.9},
  'Fries': {'Cal': 248,
   'Carbo': 30,
   'Protein': 3,
   'VitA': 0,
   'VitC': 5,
   'Calc': 7,
   'Iron': 0.6},
  'Milk': {'Cal': 138,
   'Carbo': 10,
   'Protein': 7,
   'VitA': 80,
   'VitC': 2,
   'Calc': 227,
   'Iron': 0},
  'VegJuice': {'Cal': 69,
   'Carbo': 17,
   'Protein': 1,
   'VitA': 750,
   'VitC':

In [16]:
 N, a, b

(['Cal', 'Carbo', 'Protein', 'VitA', 'VitC', 'Calc', 'Iron'],
 {'Cal': 2000,
  'Carbo': 300,
  'Protein': 50,
  'VitA': 500,
  'VitC': 85,
  'Calc': 660,
  'Iron': 6.0},
 {'Cal': 3000,
  'Carbo': 375,
  'Protein': 60,
  'VitA': 750,
  'VitC': 100,
  'Calc': 900,
  'Iron': 7.5})

In [22]:
model = Model('modern diet')
x = {}
for j in F:
    x[j] = model.addVar(vtype='C', name='x({0})'.format(j))
model.update()
for i in N:
    model.addConstr(quicksum(n[j][i]*x[j] for j in F) >= a[i], 'NutrLB({0})'.format(i))
    model.addConstr(quicksum(n[j][i]*x[j] for j in F) <= b[i], 'NutrUB({0})'.format(i))
model.setObjective(quicksum(c[j]*x[j] for j in F), GRB.MINIMIZE)
model.update()
model

modern diet:
MINIMIZE
320*x(Big_M) + 360*x(CQPounder) + 290*x(Chicken) + 270*x(FFilet) + 190*x(Fries) + 170*x(Milk) + 100*x(VegJuice) + 0
SUBJECT TO
NutrLB(Cal): 556 x(Big_M) + 556 x(CQPounder) + 431 x(Chicken) + 356 x(FFilet)
 + 248 x(Fries) + 138 x(Milk) + 69 x(VegJuice) >= 2000

NutrUB(Cal): 556 x(Big_M) + 556 x(CQPounder) + 431 x(Chicken) + 356 x(FFilet)
 + 248 x(Fries) + 138 x(Milk) + 69 x(VegJuice) <= 3000

NutrLB(Carbo): 46 x(Big_M) + 39 x(CQPounder) + 45 x(Chicken) + 42 x(FFilet)
 + 30 x(Fries) + 10 x(Milk) + 17 x(VegJuice) >= 300

NutrUB(Carbo): 46 x(Big_M) + 39 x(CQPounder) + 45 x(Chicken) + 42 x(FFilet)
 + 30 x(Fries) + 10 x(Milk) + 17 x(VegJuice) <= 375

NutrLB(Protein): 26 x(Big_M) + 30 x(CQPounder) + 20 x(Chicken) + 14 x(FFilet)
 + 3 x(Fries) + 7 x(Milk) + x(VegJuice) >= 50

NutrUB(Protein): 26 x(Big_M) + 30 x(CQPounder) + 20 x(Chicken) + 14 x(FFilet)
 + 3 x(Fries) + 7 x(Milk) + x(VegJuice) <= 60

NutrLB(VitA): 97 x(Big_M) + 147 x(CQPounder) + 9 x(Chicken) + 28 x(FFilet)


In [26]:
model.optimize()

In [25]:
model.Status == GRB.Status.OPTIMAL

False

In [27]:
F, N

(['CQPounder', 'Big M', 'FFilet', 'Chicken', 'Fries', 'Milk', 'VegJuice'],
 ['Cal', 'Carbo', 'Protein', 'VitA', 'VitC', 'Calc', 'Iron'])

In [37]:
model = Model('revised modern diet')
x, s, d = {}, {}, {}
for j in F:
    x[j] = model.addVar(vtype='C', name='x({0})'.format(j))
for i in N:
    s[i] = model.addVar(vtype='C', name='surplus({0})'.format(i))
    d[i] = model.addVar(vtype='C', name='deficit({0})'.format(i))
    
model.update()
for i in N:
    model.addConstr(quicksum(n[j][i]*x[j] for j in F) >= a[i]-d[i], 'NutrLB({0})'.format(i))
    model.addConstr(quicksum(n[j][i]*x[j] for j in F) <= b[i]+s[i], 'NutrUB({0})'.format(i))
model.setObjective(quicksum(c[j]*x[j] for j in F) + 
                   quicksum(9999*d[i] + 9999*s[i] for i in N), GRB.MINIMIZE)
model.update()
model

revised modern diet:
MINIMIZE
9999*deficit(Cal) + 9999*deficit(Calc) + 9999*deficit(Carbo) + 9999*deficit(Iron) + 9999*deficit(Protein) + 9999*deficit(VitA) + 9999*deficit(VitC) + 9999*surplus(Cal) + 9999*surplus(Calc) + 9999*surplus(Carbo) + 9999*surplus(Iron) + 9999*surplus(Protein) + 9999*surplus(VitA) + 9999*surplus(VitC) + 320*x(Big_M) + 360*x(CQPounder) + 290*x(Chicken) + 270*x(FFilet) + 190*x(Fries) + 170*x(Milk) + 100*x(VegJuice) + 0
SUBJECT TO
NutrLB(Cal): deficit(Cal) + 556 x(Big_M) + 556 x(CQPounder) + 431 x(Chicken)
 + 356 x(FFilet) + 248 x(Fries) + 138 x(Milk) + 69 x(VegJuice) >= 2000

NutrUB(Cal): - surplus(Cal) + 556 x(Big_M) + 556 x(CQPounder) + 431 x(Chicken)
 + 356 x(FFilet) + 248 x(Fries) + 138 x(Milk) + 69 x(VegJuice) <= 3000

NutrLB(Carbo): deficit(Carbo) + 46 x(Big_M) + 39 x(CQPounder) + 45 x(Chicken)
 + 42 x(FFilet) + 30 x(Fries) + 10 x(Milk) + 17 x(VegJuice) >= 300

NutrUB(Carbo): - surplus(Carbo) + 46 x(Big_M) + 39 x(CQPounder)
 + 45 x(Chicken) + 42 x(FFilet) +

In [38]:
%%time
model.optimize()

CPU times: user 4.9 ms, sys: 11.8 ms, total: 16.7 ms
Wall time: 26.4 ms


In [52]:
if model.Status == GRB.Status.OPTIMAL:
    print('ObjVal: ', model.ObjVal)
    print('\n# opt variables')
    for j in F:
        print(j, x[j].X)
    print('\n# deficit and surplus of')
    for i in N:
        print('{0}\t{1}\t{2}'.format(i, d[i].X, s[i].X))


ObjVal:  262979.603553112

# opt variables
CQPounder 0.0066833892
Big M 0.0
FFilet 0.0
Chicken 0.0
Fries 10.476435
Milk 2.520044
VegJuice 0.72988536

# deficit and surplus of
Cal	0.0	0.0
Carbo	0.0	0.0
Protein	0.0	0.0
VitA	0.0	0.0
VitC	26.051133	0.0
Calc	0.0	0.0
Iron	0.0	0.0
